In [1]:
import torch
import torchvision
import torch.nn as nn
import torch.nn.functional as F
from torchvision import datasets 
from torchvision import transforms
from torchvision.utils import save_image
from torch.autograd import Variable


import pandas as pd 
import numpy as np
import os 
import glob
np.random.seed(1337)  # for reproducibility
from sklearn import preprocessing
from sklearn.model_selection import train_test_split
import torch.optim as optim
import torch
import torch.nn.functional as nn
import torch.autograd as autograd
import torch.optim as optim
import numpy as np
#import matplotlib.pyplot as plt
#import matplotlib.gridspec as gridspec
import os
from torch.autograd import Variable
from sklearn.utils import shuffle
import random
from keras.datasets import mnist
from keras.callbacks import EarlyStopping
from keras.callbacks import ModelCheckpoint
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation, Flatten
from keras.layers import Convolution2D, MaxPooling2D
from keras.utils import np_utils
from keras import backend as K
from keras import optimizers

Using TensorFlow backend.


In [2]:
train_lre = pd.read_csv('/home/satishk/lre2.0/ivectors_csv_revised/train_feat_BNF_h5_07Nov_Shreyas.csv')
#train_afds

In [3]:
#train_lre = train_lre.iloc[1000:2000]

In [4]:
train_lre.head()

,0,1,2,3,4,5,6,7,8,9,...,491,492,493,494,495,496,497,498,499,langid
0,-0.141175,-1.160019,0.891814,1.898842,0.393065,0.983582,-0.559143,-0.761900,0.525598,0.344597,...,1.587687,-0.609223,1.529694,1.677775,-0.388426,1.044584,-1.365691,-0.354752,-0.815562,spa-eur
1,0.971834,0.035743,0.785385,1.328749,-0.043342,1.219254,0.761090,1.069464,0.879541,0.898939,...,-1.208031,0.527433,-0.709180,-1.117489,0.566520,1.642208,-0.703815,0.376027,-1.425985,spa-eur
2,-0.315703,-0.868423,0.619893,1.717784,-0.846024,1.177214,-0.191977,-0.658569,0.529625,0.313590,...,-0.896515,-0.128051,-0.075781,1.307904,-1.443047,2.551083,0.436309,-0.623076,0.194851,spa-eur
3,0.247949,-0.279720,1.320115,1.772152,-0.130394,0.517343,-0.290516,0.406225,-0.833366,1.567592,...,-0.398762,-0.078148,0.168076,-0.272796,-1.102862,0.358274,0.090417,-0.490244,1.819196,spa-eur
4,-0.986578,-0.357548,1.550832,1.400735,0.444350,0.465660,0.209248,1.175847,-1.079963,0.270915,...,0.947946,0.917957,-1.043352,0.414811,0.016396,-0.485602,-0.433002,-0.432070,0.778250,spa-eur


In [5]:
train_lre.groupby(['langid']).size()

langid
ara-acm    1406
ara-apc    3509
ara-ary     919
ara-arz     440
eng-gbr      98
eng-usg    2448
por-brz     444
qsl-pol     587
qsl-rus    1221
spa-car     688
spa-eur     121
spa-lac     898
zho-cmn    3331
zho-nan      95
dtype: int64

In [6]:
val_lre = pd.read_csv('/home/satishk/lre2.0/ivectors_csv_revised/dev_feat_BNF_h5_07Nov_Shreyas.csv')


In [7]:
#val_lre = val_lre.iloc[100:300]

In [8]:
val_lre.head()

,segmentid,language_code,data_source,speech_duration,0,1,2,3,4,5,...,491,492,493,494,495,496,497,498,499,uttid
0,lre17_ntrlosgu.sph,ara-acm,mls14,30,1.697234,0.029428,-0.400756,0.513963,-0.939232,1.500797,...,-1.314428,-0.927694,-0.370424,-0.514735,1.290885,0.688205,-0.494330,-0.053206,-1.330860,lre17_ntrlosgu
1,lre17_moxnwuqe.sph,ara-acm,mls14,10,1.648232,-0.053318,-0.562867,1.035870,-1.577741,1.593584,...,-0.929262,-1.301574,2.034934,-0.226545,-0.198926,-0.116174,0.347923,-0.870801,-2.599601,lre17_moxnwuqe
2,lre17_meesvkxz.sph,ara-acm,mls14,3,1.242829,0.675515,-0.371491,0.534970,-0.246783,0.806262,...,0.691336,0.257988,1.058771,1.018635,-1.929319,-0.307404,-0.486431,-2.839053,-2.704527,lre17_meesvkxz
3,lre17_rqmsmzui.sph,ara-acm,mls14,30,1.226681,0.014810,-0.396915,-0.097507,-0.013574,1.087025,...,1.049862,0.285627,2.385587,0.680073,1.500978,1.660566,-0.370672,-0.924109,0.096676,lre17_rqmsmzui
4,lre17_qgszpuyw.sph,ara-acm,mls14,10,1.411728,-0.119300,0.136256,0.030535,-1.029447,1.227100,...,0.155196,-1.030222,2.933880,-1.417872,-0.227513,0.748810,-0.671044,0.595977,1.722917,lre17_qgszpuyw


In [9]:
X_train = train_lre.drop("langid",axis=1)
y_train = train_lre["langid"]
#y_train_uttid = train_lre["uttid"]

In [10]:
X_val = val_lre.drop(["language_code","uttid","segmentid","data_source","speech_duration"],axis=1)
y_val = val_lre["language_code"]
y_val_segmentid = val_lre["segmentid"]

In [11]:
X_train = X_train.append(X_val, ignore_index=True)
y_train = y_train.append(y_val, ignore_index=True)

In [12]:
le = preprocessing.LabelEncoder()
le.fit(y_train)

LabelEncoder()

In [13]:
le.classes_

array(['ara-acm', 'ara-apc', 'ara-ary', 'ara-arz', 'eng-gbr', 'eng-usg',
       'por-brz', 'qsl-pol', 'qsl-rus', 'spa-car', 'spa-eur', 'spa-lac',
       'zho-cmn', 'zho-nan'], dtype=object)

In [14]:
y_train=le.transform(y_train)

In [15]:
le = preprocessing.LabelEncoder()
le.fit(y_val)

LabelEncoder()

In [16]:
le.classes_

array(['ara-acm', 'ara-apc', 'ara-ary', 'ara-arz', 'eng-gbr', 'eng-usg',
       'por-brz', 'qsl-pol', 'qsl-rus', 'spa-car', 'spa-eur', 'spa-lac',
       'zho-cmn', 'zho-nan'], dtype=object)

In [17]:
y_val_labels = le.transform(y_val)

In [18]:
y_val_labels[0:100]

array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0])

In [19]:
X_train=X_train.values
X_val=X_val.values

In [20]:
nb_classes = 14

In [21]:
def indices_to_one_hot(data, nb_classes):
    """Convert an iterable of indices to one-hot encoded labels."""
    targets = np.array(data).reshape(-1)
    return np.eye(nb_classes)[targets]

In [22]:
Y_train = indices_to_one_hot(y_train, 14)
#Y_test = indices_to_one_hot(y_test, 14)
Y_val = indices_to_one_hot(y_val_labels, 14)

In [23]:
Y_val

array([[ 1.,  0.,  0., ...,  0.,  0.,  0.],
       [ 1.,  0.,  0., ...,  0.,  0.,  0.],
       [ 1.,  0.,  0., ...,  0.,  0.,  0.],
       ..., 
       [ 0.,  0.,  0., ...,  0.,  0.,  1.],
       [ 0.,  0.,  0., ...,  0.,  0.,  1.],
       [ 0.,  0.,  0., ...,  0.,  0.,  1.]])

In [24]:
Y_train

array([[ 0.,  0.,  0., ...,  0.,  0.,  0.],
       [ 0.,  0.,  0., ...,  0.,  0.,  0.],
       [ 0.,  0.,  0., ...,  0.,  0.,  0.],
       ..., 
       [ 0.,  0.,  0., ...,  0.,  0.,  1.],
       [ 0.,  0.,  0., ...,  0.,  0.,  1.],
       [ 0.,  0.,  0., ...,  0.,  0.,  1.]])

In [25]:
Y_val.shape

(3661, 14)

In [26]:
Y_train.shape

(19866, 14)

In [27]:
X_train = X_train.astype('float32')
#X_test = X_test.astype('float32')
X_val = X_val.astype('float32')

#X_train /= 255
#X_test /= 255
#X_val /= 255

print('X_train shape:', X_train.shape)
print(X_train.shape[0], 'train samples')
#print(X_test.shape[0], 'test samples')
print(X_val.shape[0], 'val samples')


X_train shape: (19866, 500)
19866 train samples
3661 val samples


In [28]:
X_train.shape

(19866, 500)

In [29]:
y_train.shape

(19866,)

In [30]:
#X_train,  y_train = shuffle(X_train, y_train, random_state=0)

In [31]:
X_val.shape

(3661, 500)

In [32]:
y_val_labels

array([ 0,  0,  0, ..., 13, 13, 13])

In [33]:
#X_val,  y_val_labels = shuffle(X_val, y_val_labels, random_state=0)

In [34]:
X_train, X_test, Y_train, y_test = train_test_split(X_train, Y_train, test_size=0.2, random_state=42)

In [35]:
#mnist = input_data.read_data_sets('/home/satishk/depy_04_AUG/MNIST_data', one_hot=True)
mb_size = 256
Z_dim = 100
X_dim = 500 #mnist.train.images.shape[1]
y_dim = 14 #mnist.train.labels.shape[1]
h_dim = 128
cnt = 0
lr = 1e-3


In [36]:
X_dim, y_dim

(500, 14)

In [37]:
def xavier_init(size):
    in_dim = size[0]
    xavier_stddev = 1. / np.sqrt(in_dim / 2.)
    return Variable(torch.randn(*size) * xavier_stddev, requires_grad=True)

In [38]:
""" ==================== GENERATOR ======================== """

Wzh = xavier_init(size=[Z_dim + y_dim, h_dim])
bzh = Variable(torch.zeros(h_dim), requires_grad=True)

Whx = xavier_init(size=[h_dim, X_dim])
bhx = Variable(torch.zeros(X_dim), requires_grad=True)


def G(z, c):
    inputs = torch.cat([z, c], 1)
    h = nn.relu(inputs @ Wzh + bzh.repeat(inputs.size(0), 1))
    X = nn.sigmoid(h @ Whx + bhx.repeat(h.size(0), 1))
    return X



In [39]:
""" ==================== DISCRIMINATOR ======================== """

Wxh = xavier_init(size=[X_dim + y_dim, h_dim])
bxh = Variable(torch.zeros(h_dim), requires_grad=True)

Why = xavier_init(size=[h_dim, 1])
bhy = Variable(torch.zeros(1), requires_grad=True)


def D(X, c):
    inputs = torch.cat([X, c], 1)
    h = nn.relu(inputs @ Wxh + bxh.repeat(inputs.size(0), 1))
    y = nn.sigmoid(h @ Why + bhy.repeat(h.size(0), 1))
    return y


G_params = [Wzh, bzh, Whx, bhx]
D_params = [Wxh, bxh, Why, bhy]
params = G_params + D_params

In [40]:
def reset_grad():
    for p in params:
        if p.grad is not None:
            data = p.grad.data
            p.grad = Variable(data.new().resize_as_(data).zero_())


G_solver = optim.Adam(G_params, lr=1e-3)
D_solver = optim.Adam(D_params, lr=1e-3)

#ones_label = Variable(torch.ones(mb_size))
#zeros_label = Variable(torch.zeros(mb_size))

ones_label = Variable(torch.ones(mb_size))
zeros_label = Variable(torch.zeros(mb_size))


In [41]:
len(X_train)

15892

In [42]:
batch_size = mb_size
# Start training
for epoch in range(10):
    
    

    print('epoch:',epoch)
    #for i in range(XX_train):
    # Build mini-batch dataset
    #batch_size = images.size(0)
    #images = to_var(images.view(batch_size, -1))

    it=0
    while it+batch_size < len(X_train) :
        

        start= it
        end= it + batch_size


        z = Variable(torch.randn(mb_size, Z_dim))
        X = X_train[start:end]

        c = Y_train[start:end]
        X = Variable(torch.from_numpy(X))
        c = Variable(torch.from_numpy(c.astype('float32')))

        # Dicriminator forward-loss-backward-update
        G_sample = G(z, c)
        D_real = D(X, c)
        D_fake = D(G_sample, c)

        D_loss_real = nn.binary_cross_entropy(D_real, ones_label)
        D_loss_fake = nn.binary_cross_entropy(D_fake, zeros_label)
        D_loss = D_loss_real + D_loss_fake

        D_loss.backward()
        D_solver.step()

        # Housekeeping - reset gradient
        reset_grad()

        # Generator forward-loss-backward-update
        z = Variable(torch.randn(mb_size, Z_dim))
        G_sample = G(z, c)
        D_fake = D(G_sample, c)

        G_loss = nn.binary_cross_entropy(D_fake, ones_label)

        G_loss.backward()
        G_solver.step()

        # Housekeeping - reset gradient
        reset_grad()

        #Print and plot every now and then
        #if it % 2 == 0:

        print('Iter-{}; D_loss: {}; G_loss: {}'.format(it, D_loss.data.numpy(), G_loss.data.numpy()))



        it+= batch_size
           

epoch: 0
Iter-0; D_loss: [ 2.3023262]; G_loss: [ 0.65551734]
Iter-256; D_loss: [ 1.25758111]; G_loss: [ 1.44586122]
Iter-512; D_loss: [ 0.66614532]; G_loss: [ 2.40486073]
Iter-768; D_loss: [ 0.51211506]; G_loss: [ 3.29984045]
Iter-1024; D_loss: [ 0.38790327]; G_loss: [ 4.02416039]
Iter-1280; D_loss: [ 0.3340939]; G_loss: [ 4.55242443]


/home/satishk/miniconda3/envs/lre17/lib/python3.6/site-packages/torch/nn/functional.py:1168: UserWarning: Using a target size (torch.Size([256])) that is different to the input size (torch.Size([256, 1])) is deprecated. Please ensure they have the same size.
  "Please ensure they have the same size.".format(target.size(), input.size()))


Iter-1536; D_loss: [ 0.32332253]; G_loss: [ 4.98773336]
Iter-1792; D_loss: [ 0.30346888]; G_loss: [ 5.33819342]
Iter-2048; D_loss: [ 0.2884827]; G_loss: [ 5.46230412]
Iter-2304; D_loss: [ 0.25673652]; G_loss: [ 5.57792759]
Iter-2560; D_loss: [ 0.29571006]; G_loss: [ 5.64273214]
Iter-2816; D_loss: [ 0.24197854]; G_loss: [ 5.65974188]
Iter-3072; D_loss: [ 0.19153138]; G_loss: [ 5.63087082]
Iter-3328; D_loss: [ 0.20759515]; G_loss: [ 5.51941299]
Iter-3584; D_loss: [ 0.1678593]; G_loss: [ 5.46293974]
Iter-3840; D_loss: [ 0.15125678]; G_loss: [ 5.22235489]
Iter-4096; D_loss: [ 0.1410941]; G_loss: [ 5.11091948]
Iter-4352; D_loss: [ 0.14233965]; G_loss: [ 4.91631126]
Iter-4608; D_loss: [ 0.11724956]; G_loss: [ 4.77727318]
Iter-4864; D_loss: [ 0.13049422]; G_loss: [ 4.56257248]
Iter-5120; D_loss: [ 0.13021444]; G_loss: [ 4.36846733]
Iter-5376; D_loss: [ 0.10716747]; G_loss: [ 4.24481344]
Iter-5632; D_loss: [ 0.12209201]; G_loss: [ 4.01036072]
Iter-5888; D_loss: [ 0.11139617]; G_loss: [ 3.83996

Iter-7424; D_loss: [ 0.41333833]; G_loss: [ 1.33515155]
Iter-7680; D_loss: [ 0.42966643]; G_loss: [ 1.33285689]
Iter-7936; D_loss: [ 0.38023508]; G_loss: [ 1.38303411]
Iter-8192; D_loss: [ 0.45729148]; G_loss: [ 1.41162372]
Iter-8448; D_loss: [ 0.41360563]; G_loss: [ 1.41926134]
Iter-8704; D_loss: [ 0.41986912]; G_loss: [ 1.45077908]
Iter-8960; D_loss: [ 0.3530876]; G_loss: [ 1.48556125]
Iter-9216; D_loss: [ 0.34676674]; G_loss: [ 1.51643968]
Iter-9472; D_loss: [ 0.33913654]; G_loss: [ 1.55382049]
Iter-9728; D_loss: [ 0.32378763]; G_loss: [ 1.57969856]
Iter-9984; D_loss: [ 0.28817326]; G_loss: [ 1.6194886]
Iter-10240; D_loss: [ 0.30501205]; G_loss: [ 1.65032136]
Iter-10496; D_loss: [ 0.29206944]; G_loss: [ 1.67248964]
Iter-10752; D_loss: [ 0.28468227]; G_loss: [ 1.70386136]
Iter-11008; D_loss: [ 0.29064399]; G_loss: [ 1.72611129]
Iter-11264; D_loss: [ 0.27787703]; G_loss: [ 1.76992452]
Iter-11520; D_loss: [ 0.28045863]; G_loss: [ 1.77770567]
Iter-11776; D_loss: [ 0.2676726]; G_loss: [ 

Iter-14592; D_loss: [ 0.05335689]; G_loss: [ 3.2454989]
Iter-14848; D_loss: [ 0.05300501]; G_loss: [ 3.25839257]
Iter-15104; D_loss: [ 0.05304332]; G_loss: [ 3.28762126]
Iter-15360; D_loss: [ 0.0532539]; G_loss: [ 3.29980111]
Iter-15616; D_loss: [ 0.05761059]; G_loss: [ 3.32443142]
epoch: 5
Iter-0; D_loss: [ 0.0491818]; G_loss: [ 3.32456136]
Iter-256; D_loss: [ 0.04486523]; G_loss: [ 3.35599756]
Iter-512; D_loss: [ 0.0482597]; G_loss: [ 3.36574006]
Iter-768; D_loss: [ 0.0452049]; G_loss: [ 3.38436079]
Iter-1024; D_loss: [ 0.04421973]; G_loss: [ 3.39307356]
Iter-1280; D_loss: [ 0.04952749]; G_loss: [ 3.371135]
Iter-1536; D_loss: [ 0.04858022]; G_loss: [ 3.36069989]
Iter-1792; D_loss: [ 0.04785639]; G_loss: [ 3.37188411]
Iter-2048; D_loss: [ 0.05018205]; G_loss: [ 3.38671422]
Iter-2304; D_loss: [ 0.04817586]; G_loss: [ 3.32293534]
Iter-2560; D_loss: [ 0.05042559]; G_loss: [ 3.36796474]
Iter-2816; D_loss: [ 0.04835045]; G_loss: [ 3.36879516]
Iter-3072; D_loss: [ 0.04520224]; G_loss: [ 3.3

Iter-4352; D_loss: [ 0.01137492]; G_loss: [ 4.94225073]
Iter-4608; D_loss: [ 0.01077524]; G_loss: [ 4.94183207]
Iter-4864; D_loss: [ 0.01111945]; G_loss: [ 4.91877127]
Iter-5120; D_loss: [ 0.01219908]; G_loss: [ 4.89935398]
Iter-5376; D_loss: [ 0.01073392]; G_loss: [ 4.90966606]
Iter-5632; D_loss: [ 0.010571]; G_loss: [ 4.95287848]
Iter-5888; D_loss: [ 0.01254489]; G_loss: [ 4.9075799]
Iter-6144; D_loss: [ 0.0116777]; G_loss: [ 4.95284081]
Iter-6400; D_loss: [ 0.010719]; G_loss: [ 4.97396708]
Iter-6656; D_loss: [ 0.01208604]; G_loss: [ 4.98453903]
Iter-6912; D_loss: [ 0.01193361]; G_loss: [ 4.984303]
Iter-7168; D_loss: [ 0.01029903]; G_loss: [ 5.00475788]
Iter-7424; D_loss: [ 0.01213145]; G_loss: [ 4.99706984]
Iter-7680; D_loss: [ 0.01116285]; G_loss: [ 4.98260355]
Iter-7936; D_loss: [ 0.01092865]; G_loss: [ 4.99696732]
Iter-8192; D_loss: [ 0.01380358]; G_loss: [ 5.02067232]
Iter-8448; D_loss: [ 0.01204208]; G_loss: [ 4.97761536]
Iter-8704; D_loss: [ 0.01251659]; G_loss: [ 4.93849421]


Iter-12032; D_loss: [ 0.0023589]; G_loss: [ 7.46758747]
Iter-12288; D_loss: [ 0.00142409]; G_loss: [ 7.4667263]
Iter-12544; D_loss: [ 0.00280147]; G_loss: [ 7.49060297]
Iter-12800; D_loss: [ 0.00162919]; G_loss: [ 7.49815941]
Iter-13056; D_loss: [ 0.00247647]; G_loss: [ 7.51520157]
Iter-13312; D_loss: [ 0.00143332]; G_loss: [ 7.47885609]
Iter-13568; D_loss: [ 0.00239291]; G_loss: [ 7.49562883]
Iter-13824; D_loss: [ 0.00173712]; G_loss: [ 7.43355989]
Iter-14080; D_loss: [ 0.00240493]; G_loss: [ 7.42631531]
Iter-14336; D_loss: [ 0.0016101]; G_loss: [ 7.39134359]
Iter-14592; D_loss: [ 0.00169592]; G_loss: [ 7.32188892]
Iter-14848; D_loss: [ 0.00240667]; G_loss: [ 7.25284767]
Iter-15104; D_loss: [ 0.00237612]; G_loss: [ 7.17829227]
Iter-15360; D_loss: [ 0.00174953]; G_loss: [ 7.14638662]
Iter-15616; D_loss: [ 0.00262559]; G_loss: [ 7.07986069]


In [43]:
df_SF=pd.DataFrame()
samples_per_class = 1000
#c = np.zeros(shape=[samples_per_class, y_dim], dtype='float32')
#c[:, np.random.randint(0, 10)] = 1.
for i in range(14):
    #print(i)
    c = np.zeros(shape=[samples_per_class, y_dim], dtype='float32')
    c[:, i] = 1.
    c_df=pd.DataFrame(c)
    df_SF = df_SF.append(c_df,ignore_index = True)
    
    

In [44]:
c_df.shape

(1000, 14)

In [45]:
df_SF.shape[0]

14000

In [46]:
c_gen = df_SF.values

In [47]:
z_gen = Variable(torch.randn(df_SF.shape[0], Z_dim))

In [48]:
z_gen.data.shape

torch.Size([14000, 100])

In [49]:
c_gen = Variable(torch.from_numpy(c_gen))

In [50]:
samples = G(z_gen, c_gen)

In [51]:
samples.data.shape

torch.Size([14000, 500])

In [52]:
#Taking the generated iVectors we will try to check the acc by MLP

In [53]:
train_X1 = samples.data.numpy()

In [54]:
type(train_X1)

numpy.ndarray

In [55]:
train_y1 = c_gen.data.numpy()

In [56]:
type(train_y1)

numpy.ndarray

In [57]:
train_y1

array([[ 1.,  0.,  0., ...,  0.,  0.,  0.],
       [ 1.,  0.,  0., ...,  0.,  0.,  0.],
       [ 1.,  0.,  0., ...,  0.,  0.,  0.],
       ..., 
       [ 0.,  0.,  0., ...,  0.,  0.,  1.],
       [ 0.,  0.,  0., ...,  0.,  0.,  1.],
       [ 0.,  0.,  0., ...,  0.,  0.,  1.]], dtype=float32)

In [58]:
X_train = pd.DataFrame(X_train)
Y_train = pd.DataFrame(Y_train)


In [59]:
train_X1 = pd.DataFrame(train_X1)
train_y1 = pd.DataFrame(train_y1)

In [60]:
#train_y

In [61]:
train_X = X_train.append(train_X1, ignore_index=True)
train_y = Y_train.append(train_y1, ignore_index=True)

In [62]:
#train_X,  train_y = shuffle(train_X, train_y, random_state=0)

In [63]:
train_X = train_X.values
train_y = train_y.values

In [64]:
train_y

array([[ 0.,  0.,  1., ...,  0.,  0.,  0.],
       [ 0.,  0.,  0., ...,  0.,  0.,  0.],
       [ 0.,  0.,  0., ...,  0.,  0.,  0.],
       ..., 
       [ 0.,  0.,  0., ...,  0.,  0.,  1.],
       [ 0.,  0.,  0., ...,  0.,  0.,  1.],
       [ 0.,  0.,  0., ...,  0.,  0.,  1.]])

In [65]:
model = Sequential()
model.add(Dense(512, input_dim=500))
model.add(Activation('relu'))
model.add(Dropout(0.5))

model.add(Dense(512))
model.add(Activation('relu'))
model.add(Dropout(0.5))

model.add(Dense(nb_classes))
model.add(Activation('softmax'))

In [66]:
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_1 (Dense)              (None, 512)               256512    
_________________________________________________________________
activation_1 (Activation)    (None, 512)               0         
_________________________________________________________________
dropout_1 (Dropout)          (None, 512)               0         
_________________________________________________________________
dense_2 (Dense)              (None, 512)               262656    
_________________________________________________________________
activation_2 (Activation)    (None, 512)               0         
_________________________________________________________________
dropout_2 (Dropout)          (None, 512)               0         
_________________________________________________________________
dense_3 (Dense)              (None, 14)                7182      
__________

In [67]:
adam=optimizers.Adam(lr=0.001, beta_1=0.9, beta_2=0.999, epsilon=1e-08, decay=0.0)

model.compile(loss='categorical_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])

In [68]:
best_weights_filepath = '/home/satishk/saved_weights/best_weights_2l_MLP_11.hdf5'
saveBestModel = ModelCheckpoint(best_weights_filepath, monitor='val_acc', verbose=1, save_best_only=True, mode='auto')


In [69]:
batch_size = 256
nb_epoch=30

In [71]:
#Checking Accuracy with training+augmented data train_X and train_y are 'train + augmented' data
history = model.fit(train_X, train_y, batch_size=batch_size, epochs=nb_epoch,verbose=2, 
                    validation_data=(X_test , y_test),callbacks=[saveBestModel])

Train on 14000 samples, validate on 3974 samples
Epoch 1/30
Epoch 00000: val_acc improved from -inf to 0.05611, saving model to /home/satishk/saved_weights/best_weights_2l_MLP_11.hdf5
1s - loss: 2.6537 - acc: 0.0699 - val_loss: 2.8248 - val_acc: 0.0561
Epoch 2/30
Epoch 00001: val_acc improved from 0.05611 to 0.06241, saving model to /home/satishk/saved_weights/best_weights_2l_MLP_11.hdf5
1s - loss: 2.6419 - acc: 0.0701 - val_loss: 2.7979 - val_acc: 0.0624
Epoch 3/30
Epoch 00002: val_acc did not improve
1s - loss: 2.6405 - acc: 0.0715 - val_loss: 2.8027 - val_acc: 0.0579
Epoch 4/30
Epoch 00003: val_acc did not improve
1s - loss: 2.6396 - acc: 0.0751 - val_loss: 2.8220 - val_acc: 0.0544
Epoch 5/30
Epoch 00004: val_acc did not improve
1s - loss: 2.6399 - acc: 0.0707 - val_loss: 2.8020 - val_acc: 0.0554
Epoch 6/30
Epoch 00005: val_acc did not improve
1s - loss: 2.6395 - acc: 0.0721 - val_loss: 2.7953 - val_acc: 0.0591
Epoch 7/30
Epoch 00006: val_acc improved from 0.06241 to 0.06417, saving

In [72]:
#Frame label accuracy
score = model.evaluate(X_test, y_test, verbose=0)
print('ERROR after Data Augmentation %:', 1-score[1])

ERROR after Data Augmentation %: 0.908152994464
